In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/handout
%cd /content/drive/MyDrive/handout/hw4

In [ ]:
!unzip /content/handout.zip

In [ ]:
%cd /content/handout
%cd /content/handout/hw4

In [ ]:
! pip install torchsummaryX

In [1]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import time
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from tests_hw4 import test_prediction, test_generation
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [2]:
from torchsummaryX import summary

In [3]:
# load all that we need
dataset = np.load('../dataset/wiki.train.npy', allow_pickle=True)
devset = np.load('../dataset/wiki.valid.npy', allow_pickle=True)
fixtures_pred = np.load('../fixtures/prediction.npz')  # dev
fixtures_gen = np.load('../fixtures/generation.npy')  # dev
fixtures_pred_test = np.load('../fixtures/prediction_test.npz')  # test
fixtures_gen_test = np.load('../fixtures/generation_test.npy')  # test
vocab = np.load('../dataset/vocab.npy')

In [4]:
# data loader
class DataLoaderForLanguageModeling(DataLoader):
    """
        TODO: Define data loader logic here
    """
    def __init__(self, dataset, batch_size, shuffle=True):
        self.dataset = dataset# TODO
        self.batch_size = batch_size# TODO
        self.shuffle = shuffle# TODO
        self.bptt_prob = 0.95
        # raise NotImplemented

    def __iter__(self):
        """
            You may implement some of the techniques in https://arxiv.org/pdf/1708.02182.pdf
            example: Variable length backpropagation sequences (Section 4.1)
        """
        # 1. Randomly shuffle all the articles from the WikiText-2 dataset.
        if self.shuffle:
            np.random.shuffle(self.dataset)
        # 2. Concatenate all text in one long string.
        data = np.concatenate(self.dataset)
        # select bptt seq_length for each epoch
        seq_p = np.random.random_sample()
        if seq_p > self.bptt_prob: 
            self.seq_len = int(np.random.normal(35, 5))
        else: 
            self.seq_len = int(np.random.normal(70, 5))
        batch_number = (len(data)-1) // self.seq_len
        # 3. Group the sequences into batches.
        x = torch.LongTensor(data[:self.seq_len * batch_number]).reshape(-1, self.seq_len)
        y = torch.LongTensor(data[1:self.seq_len * batch_number+1]).reshape(-1, self.seq_len)
        # 4. Run a loop that returns a tuple of (input, label) on every iteration with yield.
        Index = 0
        while Index < batch_number:
            inputs = x[Index: Index + self.batch_size, :]
            targets = y[Index: Index + self.batch_size, :]
            Index += self.batch_size
            yield(inputs, targets)

In [5]:
class LockedDropout(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x, dropout=0.5):
        if not self.training or not dropout:
            return x
        mask = x.new_empty(1, x.size(1), x.size(2), requires_grad=False).bernoulli_(1 - dropout)
        mask = mask.div_(1 - dropout)
        mask = mask.expand_as(x)
        return mask * x

In [16]:
# model
class Model(nn.Module):
    """
        TODO: Define your model here
        https://arxiv.org/abs/1708.02182
        https://github.com/salesforce/awd-lstm-lm
    """
    def __init__(self, Dropout, vocab_size:int, embedding_dim:int, hidden_size:int, num_layers:int):
        super(Model, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers
        self.output_size = vocab_size
        self.weight_tying = True
        self.hidden_size = hidden_size #if self.weight_tying else 128
        #[0.4,0.3,0.4,0.3]
        self.dropouti = 0.5
        self.dropouth = 0.3
        self.dropout = 0.4
        self.dropoute = 0.5
        self.bidirectional = False

        self.lockdropout = Dropout()
        self.encode = torch.nn.Embedding(num_embeddings = self.vocab_size, 
                                         embedding_dim = self.embedding_dim)
        # self.rnn = []
        # for l in range(self.num_layers):
        #     if (l == 0):
        #         self.rnn.append(nn.LSTM(input_size = self.embedding_dim, hidden_size = self.hidden_size))
        #     # elif self.weight_tying and l == self.num_layers-1:
        #     #     self.rnn.append(nn.LSTM(input_size = self.hidden_size, hidden_size = self.embedding_dim))
        #     else:
        #         self.rnn.append(nn.LSTM(input_size = self.hidden_size, hidden_size = self.hidden_size))
        # self.rnn = nn.ModuleList(self.rnn)
        self.lstm = nn.LSTM(embedding_dim, self.hidden_size, num_layers= 3,
                           batch_first = False)
        self.decode = nn.Linear(self.hidden_size, self.output_size)
        # raise NotImplemented
        # if self.weight_tying:
        #     self.encode.weight = self.decode.weight

    #     self.init_weights()
    #     #initial weights
    # def init_weights(self):
    #     self.encode.weight.data.uniform_(-0.1, 0.1)
    #     self.decode.bias.data.fill_(0)
    #     self.decode.weight.data.uniform_(-0.1, 0.1)
    
    # def embedded_dropout(self, embed, words, dropout=0.1, scale=None):
    #     if dropout:
    #         mask = embed.weight.data.new().resize_((embed.weight.size(0), 1)).bernoulli_(1 - dropout).expand_as(embed.weight) / (1 - dropout)
    #         masked_embed_weight = mask * embed.weight
    #     else:
    #         masked_embed_weight = embed.weight
        
    #     if scale:
    #         masked_embed_weight = scale.expand_as(masked_embed_weight) * masked_embed_weight

    #     padding_idx = embed.padding_idx
    #     if padding_idx is None:
    #         padding_idx = -1

    #     X = torch.nn.functional.embedding(words, masked_embed_weight, padding_idx, embed.max_norm, 
    #                                       embed.norm_type, embed.scale_grad_by_freq, embed.sparse)
    #     return X
    

    def forward(self, x):
        # Feel free to add extra arguments to forward (like an argument to pass in the hiddens)
        x = x.transpose(0,1)
        embed = self.encode(x)
        embed = self.lockdropout(embed, 0.5)
        out, hidden = self.lstm(embed)
        out = self.lockdropout(out, 0.5)
        out = out.permute(1, 0, 2).contiguous()
        out = self.decode(out)
        return out
        # x = x.transpose(0,1) # Batch x Seq_len ---> Seq_len x Batch, L = sequence length, N = batch size, LxN
        # batch_size = x.size(1)
        # embed = self.embedded_dropout(self.encode, x, dropout = self.dropoute if self.training else 0)
        # if self.training and self.dropouti:
        #     embed = self.lockdropout(embed, self.dropouti)
        # rnn_inputs = embed # L x N x embedding_size
        # rnn_hidden = self.init_hidden(batch_size)
        # for l, rnn_layer in enumerate(self.rnn):
        #     rnn_outputs, _ = rnn_layer(rnn_inputs, rnn_hidden[l]) #(L, N, Hidden_size)
        #     if self.training and self.dropouth and l != self.num_layers-1:
        #         rnn_outputs = self.lockdropout(rnn_outputs, self.dropouth)
        #     rnn_inputs = rnn_outputs
        # raw_outputs = rnn_outputs
        # if self.training and self.dropoute:
        #     rnn_outputs = self.lockdropout(rnn_outputs, self.dropoute)
        # reg_outputs = rnn_outputs
        # rnn_outputs = rnn_outputs.permute(1, 0, 2).contiguous() #(N, L, H)
        # outputs = self.decode(rnn_outputs) #(N, L, vocab)(B, T, V)
        # # outputs = linear_outputs.permute(1, 0, 2).contiguous() #(L, N, vocab)(T, B, V)
        # return outputs, raw_outputs, reg_outputs
        
        # raise NotImplemented

In [17]:
# model trainer
class Trainer:
    def __init__(self, model, loader, max_epochs=1, run_id='exp',
                ):
        """
            Use this class to train your model
        """
        # feel free to add any other parameters here
        self.model = model
        self.loader = loader
        self.train_losses = []
        self.val_losses = []
        self.predictions = []
        self.predictions_test = []
        self.generated_logits = []
        self.generated = []
        self.generated_logits_test = []
        self.generated_test = []
        self.epochs = 0
        self.max_epochs = max_epochs
        self.run_id = run_id
        
        # self.lr = 5.0
        # self.weight_decay = 1.2e-6
        # TODO: Define your optimizer and criterion here
        # feel free to define a learning rate scheduler as well if you want
        # self.optimizer = torch.optim.SGD(self.model.parameters(), lr=0.1, momentum=0.9)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = 0.002, weight_decay=1e-5) 
        self.criterion = torch.nn.CrossEntropyLoss().to(device)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, patience=3, factor = 0.5)
        

    def train(self):
        self.model.train() # set to training mode
        # batch_bar = tqdm(total = len(self.loader), dynamic_ncols = True, leave = False, position = 0, desc = 'Train')
        epoch_loss = 0
        num_batches = 0
        # # Set initial hidden and cell states
        for batch_num, (inputs, targets) in enumerate(self.loader):
            epoch_loss += self.train_batch(inputs, targets)
            # if batch_num % 100 == 0:
            #     print('[TRAIN]   Loss: %.4f'
            #           % (epoch_loss/(batch_num+1)))
        epoch_loss = epoch_loss / (batch_num + 1)
        print('[TRAIN]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs + 1, self.max_epochs, epoch_loss))
        self.train_losses.append(epoch_loss)
        # self.scheduler.step()

    def detach(self, states):
        return [state.detach() for state in states]

    def train_batch(self, inputs, targets):
        """ 
            TODO: Define code for training a single batch of inputs
            
            :return 
                    (float) loss value
        """
        inputs = inputs.to(device)
        targets = targets.to(device) # B, T 
        out= self.model(inputs) #(B, T, V)
        loss = self.criterion(out.view(-1, out.size(2)), targets.view(-1))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss
        # raise NotImplemented

    
    def test(self):
        # don't change these
        self.model.eval() # set to eval mode
        predictions = TestLanguageModel.predict(fixtures_pred['inp'], self.model) # get predictions
        self.predictions.append(predictions)
        generated_logits = TestLanguageModel.generate(fixtures_gen, 10, self.model) # generated predictions for 10 words
        generated_logits_test = TestLanguageModel.generate(fixtures_gen_test, 10, self.model)
        nll = test_prediction(predictions, fixtures_pred['out'])
        generated = test_generation(fixtures_gen, generated_logits, vocab)
        generated_test = test_generation(fixtures_gen_test, generated_logits_test, vocab)
        self.val_losses.append(nll)
        
        self.generated.append(generated)
        self.generated_test.append(generated_test)
        self.generated_logits.append(generated_logits)
        self.generated_logits_test.append(generated_logits_test)
        
        # generate predictions for test data
        predictions_test = TestLanguageModel.predict(fixtures_pred_test['inp'], self.model) # get predictions
        self.predictions_test.append(predictions_test)
            
        print('[VAL]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs + 1, self.max_epochs, nll))
        self.epochs += 1

        return nll

    def save(self):
        # don't change these
        model_path = os.path.join('experiments', self.run_id, 'model-{}.pkl'.format(self.epochs))
        torch.save({'state_dict': self.model.state_dict()},
            model_path)
        np.save(os.path.join('experiments', self.run_id, 'predictions-{}.npy'.format(self.epochs)), self.predictions[-1])
        np.save(os.path.join('experiments', self.run_id, 'predictions-test-{}.npy'.format(self.epochs)), self.predictions_test[-1])
        np.save(os.path.join('experiments', self.run_id, 'generated_logits-{}.npy'.format(self.epochs)), self.generated_logits[-1])
        np.save(os.path.join('experiments', self.run_id, 'generated_logits-test-{}.npy'.format(self.epochs)), self.generated_logits_test[-1])
        with open(os.path.join('experiments', self.run_id, 'generated-{}.txt'.format(self.epochs)), 'w',  encoding='utf-8') as fw:
            fw.write(self.generated[-1])
        with open(os.path.join('experiments', self.run_id, 'generated-{}-test.txt'.format(self.epochs)), 'w', encoding='utf-8') as fw:
            fw.write(self.generated_test[-1])
        # fw = codecs.open(os.path.join('experiments', self.run_id, 'generated-{}.txt'.format(self.epochs)), 'w')
        # fw.write(self.generated_test[-1])


In [18]:
class TestLanguageModel:
    def predict(inp, model):
        """
            TODO: write prediction code here
            
            :param inp:
            :return: a np.ndarray of logits
        """
        model.eval()
        # Test the model
        inp = torch.LongTensor(inp).to(device)
        pred= model(inp) # B, T ,vocab
        pred = pred.transpose(0,1)[-1] # T, B, vocab---> B vocab
        return pred.cpu().detach().numpy()
        # raise NotImplemented

        
    def generate(inp, forward, model):
        """
            TODO: write generation code here

            Generate a sequence of words given a starting sequence.
            :param inp: Initial sequence of words (batch size, length)
            :param forward: number of additional words to generate
            :return: generated words (batch size, forward)
        """
        model.eval()
        inp = torch.LongTensor(inp).to(device)
        with torch.set_grad_enabled(False):
            sequence = []
            out= model(inp) # B, T, vocab
            currword = torch.argmax(out, dim=2)[:,-1].unsqueeze(1) # B, T
            sequence.append(currword.cpu().detach())
            if forward > 1:
                for i in range(forward - 1):
                    out= model(currword)
                    currword = torch.argmax(out, dim=2)[:,-1].unsqueeze(1) # B, T
                    sequence.append(currword.cpu().detach())
        return torch.cat(sequence, dim=1).cpu().detach().numpy()
        # model.eval()
        # inp = torch.LongTensor(inp).to(device)
        # with torch.set_grad_enabled(False):
        #     sequence = []
        #     out, _, _ = model(inp) # B, T, vocab
        #     currword = torch.argmax(out, dim=2)#[:,-1].unsqueeze(1) # B, T
        #     sequence.append(currword.cpu().detach())
        #     if forward > 1:
        #         for i in range(forward - 1):
        #             out, _, _ = model(currword)
        #             currword = torch.argmax(out, dim=2)#[:,-1].unsqueeze(1) # B, T
        #             sequence.append(currword.cpu().detach())
        # return torch.cat(sequence, dim=1).cpu().detach().numpy()
        

In [19]:
# TODO: define other hyperparameters here

NUM_EPOCHS = 80
BATCH_SIZE = 80
EMB_DIM = 400
HIDDEN_SIZE = 1150
HIDDEN_LAYERS = 3


In [10]:
run_id = str(int(time.time()))
if not os.path.exists('./experiments'):
    os.mkdir('./experiments')
os.mkdir('./experiments/%s' % run_id)
print("Saving models, predictions, and generated words to ./experiments/%s" % run_id)

Saving models, predictions, and generated words to ./experiments/1669353590


In [20]:
model = Model(Dropout = LockedDropout, vocab_size= len(vocab), embedding_dim=EMB_DIM, hidden_size=HIDDEN_SIZE, num_layers=HIDDEN_LAYERS).to(device)
# model.apply(init_weights)
loader = DataLoaderForLanguageModeling(
    dataset=dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)
trainer = Trainer(
    model=model, 
    loader=loader, 
    max_epochs=NUM_EPOCHS, 
    run_id=run_id
)

In [21]:
for inputs, target in loader:
    break

In [22]:
summary(model,inputs.to(device))

                Kernel Shape     Output Shape      Params Mult-Adds
Layer                                                              
0_encode        [400, 33278]    [26, 80, 400]    13.3112M  13.3112M
1_lockdropout              -    [26, 80, 400]           -         -
2_lstm                     -   [26, 80, 1150]    28.3176M    28.29M
3_lockdropout              -   [26, 80, 1150]           -         -
4_decode       [1150, 33278]  [80, 26, 33278]  38.302978M  38.2697M
---------------------------------------------------------------------
                          Totals
Total params          79.931778M
Trainable params      79.931778M
Non-trainable params         0.0
Mult-Adds               79.8709M


d:\anaconda3\lib\site-packages\torchsummaryX\torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_encode,"[400, 33278]","[26, 80, 400]",13311200.0,13311200.0
1_lockdropout,-,"[26, 80, 400]",NaN,NaN
2_lstm,-,"[26, 80, 1150]",28317600.0,28290000.0
3_lockdropout,-,"[26, 80, 1150]",NaN,NaN
4_decode,"[1150, 33278]","[80, 26, 33278]",38302978.0,38269700.0


In [23]:
import gc
gc.collect()

1347

In [24]:
best_nll = 1e30 
for epoch in range(NUM_EPOCHS):
    trainer.train()
    print(trainer.optimizer.param_groups[0]['lr'])
    nll = trainer.test()
    trainer.scheduler.step(nll)
    if nll < best_nll:
        best_nll = nll
        print("Saving model, predictions and generated output for epoch "+str(epoch)+" with NLL: "+ str(best_nll))
        trainer.save()
    

[TRAIN]  Epoch [1/80]   Loss: 6.7914
0.002
[VAL]  Epoch [1/80]   Loss: 5.5629
Saving model, predictions and generated output for epoch 0 with NLL: 5.5628934
[TRAIN]  Epoch [2/80]   Loss: 6.0443
0.002
[VAL]  Epoch [2/80]   Loss: 5.1450
Saving model, predictions and generated output for epoch 1 with NLL: 5.1450415
[TRAIN]  Epoch [3/80]   Loss: 5.8123
0.002
[VAL]  Epoch [3/80]   Loss: 4.8758
Saving model, predictions and generated output for epoch 2 with NLL: 4.8758135
[TRAIN]  Epoch [4/80]   Loss: 5.6004
0.002
[VAL]  Epoch [4/80]   Loss: 4.8060
Saving model, predictions and generated output for epoch 3 with NLL: 4.80601
[TRAIN]  Epoch [5/80]   Loss: 5.4825
0.002
[VAL]  Epoch [5/80]   Loss: 4.6623
Saving model, predictions and generated output for epoch 4 with NLL: 4.6623487
[TRAIN]  Epoch [6/80]   Loss: 5.3694
0.002
[VAL]  Epoch [6/80]   Loss: 4.6575
Saving model, predictions and generated output for epoch 5 with NLL: 4.6574974
[TRAIN]  Epoch [7/80]   Loss: 5.2597
0.002
[VAL]  Epoch [7/8

1669331706 #28  5.7986/4.399625

1669339744  #34,  [0.4,0.3,0.4,0.3]

1669348804  #     [0.5,0.3,0.4,0.5]



In [ ]:
# Don't change these
# plot training curves
plt.figure()
plt.plot(range(1, trainer.epochs + 1), trainer.train_losses, label='Training losses')
plt.plot(range(1, trainer.epochs + 1), trainer.val_losses, label='Validation losses')
plt.xlabel('Epochs')
plt.ylabel('NLL')
plt.legend()
plt.show()

In [ ]:
# see generated output
print (trainer.generated[-1]) # get last generated output